<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_AAI_GEOLOGY_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mistralai -q
!pip install colab-env -q
import colab_env

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.8/381.8 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Mounted at /content/gdrive


In [2]:
import os
import math
import random
from datetime import datetime, timedelta
from mistralai import Mistral

# --- Simulated Knowledge Bases for Geological Domain ---
# Instead of aircraft, we have geological survey equipment/vehicles
EQUIPMENT_DATA = {
    'DRILLING RIG (LARGE)': {'speed_deployment_kmh': 5, 'resource_consumption_rate_units_hr': 500, 'operational_range_km': 5000, 'optimal_depth_m': 3000},
    'SEISMIC VESSEL': {'speed_deployment_kmh': 20, 'resource_consumption_rate_units_hr': 1000, 'operational_range_km': 10000, 'optimal_depth_m': 2000},
    'FIELD VEHICLE (4x4)': {'speed_deployment_kmh': 40, 'resource_consumption_rate_units_hr': 50, 'operational_range_km': 1000, 'optimal_depth_m': 10},
    'UAV (SURVEY)': {'speed_deployment_kmh': 60, 'resource_consumption_rate_units_hr': 5, 'operational_range_km': 200, 'optimal_depth_m': 100}, # Altitude for aerial surveys
    'GEOPHYSICAL ROVER': {'speed_deployment_kmh': 10, 'resource_consumption_rate_units_hr': 100, 'operational_range_km': 500, 'optimal_depth_m': 50}
}

# Instead of airports, we have geological sites/areas
GEOLOGICAL_SITE_DATA = {
    'GA_RKI': {'name': 'Rocky Mountains Geological Area', 'lat': 39.00, 'lon': -106.00, 'terrain': 'Mountainous, High Altitude'},
    'GS_GTL': {'name': 'Great Lakes Basin Survey Zone', 'lat': 44.00, 'lon': -80.00, 'terrain': 'Flat, Lake-dominated'},
    'GD_SLF': {'name': 'Sahara Desert Field Site', 'lat': 25.00, 'lon': 5.00, 'terrain': 'Arid, Sandy'},
    'GR_RFN': {'name': 'Rainforest Research Node', 'lat': -5.00, 'lon': -55.00, 'terrain': 'Dense Forest, Humid'},
    'GM_OCN': {'name': 'Mid-Ocean Ridge Mapping Zone', 'lat': 0.00, 'lon': 0.00, 'terrain': 'Deep Ocean, Volcanic'},
    'AR_ICE': {'name': 'Arctic Ice Core Drilling Site', 'lat': 75.00, 'lon': 100.00, 'terrain': 'Icy, Extreme Cold'},
    'VL_JPN': {'name': 'Japanese Volcanic Arc Study Area', 'lat': 36.00, 'lon': 138.00, 'terrain': 'Volcanic, Seismically Active'},
    'RS_SCL': {'name': 'South China Limestone Karst Region', 'lat': 23.00, 'lon': 107.00, 'terrain': 'Karst, Subtropical'},
    'MZ_MID': {'name': 'Mid-Continent Rift Zone', 'lat': 45.00, 'lon': -90.00, 'terrain': 'Temperate Forest, Sedimentary'},
    'DS_SWM': {'name': 'Southwestern Desert Seismic Area', 'lat': 33.00, 'lon': -112.00, 'terrain': 'Arid, Rocky'},
    'CR_GRN': {'name': 'Greenland Cryosphere Research Site', 'lat': 69.00, 'lon': -49.00, 'terrain': 'Glacial, Remote'}
}

# --- ESOCC-Oriented Agent Roles ---
ESOCC_AGENT_ROLES = {
    'survey_request_processor': "You are an ESOCC survey request intake specialist. Your role is to gather and clarify initial geological survey requirements, identifying key scientific objectives and logistical data points.",
    'equipment_resources_agent': "You are an ESOCC equipment resource manager. Provide detailed performance and availability data for specific geological equipment/vehicles (e.g., drilling rigs, seismic vessels), including deployment speed, resource consumption rates, and operational range.",
    'site_ops_agent': "You are an ESOCC geological site operations specialist. Provide comprehensive details on geological site conditions, accessibility, local infrastructure, and any relevant environmental sensitivities.",
    'survey_planner_agent': "You are an ESOCC geological survey planner. Your expertise is in calculating optimal survey paths, logistical distances, and estimated operational durations. Consider terrain, geological features, and safety zones.",
    'geohazard_conditions_desk': "You are an ESOCC geohazard meteorologist. Analyze current and forecasted seismic activity, weather phenomena, or other environmental conditions at the origin site, providing operational implications for mobilization.",
    'target_env_conditions_desk': "You are an ESOCC environmental conditions specialist for target sites. Analyze current and forecasted environmental conditions (e.g., weather, hydrological, terrain stability) at the target geological site. Discuss operational implications for survey execution and potential hazards.",
    'enroute_logistics_env_desk': "You are an ESOCC en-route logistics and environmental specialist. Analyze potential en-route conditions (e.g., road accessibility, weather fronts) for equipment and personnel transit, identifying any significant logistical or environmental challenges.",
    'safety_env_compliance_officer': "You are an ESOCC safety and environmental compliance expert. Identify and ensure adherence to all relevant local, national, and international safety, environmental protection, and land access regulations for geological surveys.",
    'resource_logistics_optimiser': "You are an ESOCC resource and logistics planning engineer. Calculate precise resource requirements (e.g., fuel, water, specialized materials) and personnel deployment. Provide guidance on equipment load balancing and supply chain optimization.",
    'contingency_response_specialist': "You are an ESOCC contingency response specialist. Develop comprehensive alternative plans for unforeseen circumstances during surveys, such as equipment breakdowns, extreme weather, or site access issues. Identify suitable fallback locations or alternative strategies.",
    'stakeholder_liaison': "You are an ESOCC stakeholder and community liaison. Provide insights into managing relationships with local communities, indigenous groups, and other stakeholders for the survey, including communication strategies and potential impacts.",
    'personnel_scheduler': "You are an ESOCC personnel scheduler. Provide team availability, duty time limitations, rest requirements, and optimal team pairings for a given survey, ensuring compliance with labor laws and field safety protocols.",
    'equipment_readiness_coordinator': "You are an ESOCC equipment readiness coordinator. Report on geological equipment maintenance status, recent servicing, potential mechanical issues, and required calibrations/inspections to confirm readiness for deployment.",
    'geo_operational_director': "You are the ESOCC Geo-Operational Director. Your role is to synthesize all information provided by various specialized agents into a cohesive, comprehensive, and actionable geological survey plan. Highlight critical aspects, potential challenges, and key recommendations."
}

# --- ToolService Class ---
class ToolService:
    def __init__(self):
        pass

    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """Calculates the great-circle distance between two geographical points (sites) in km."""
        R = 6371.0  # Radius of Earth in kilometers
        dLat = math.radians(lat2 - lat1)
        dLon = math.radians(lon2 - lon1)
        a = math.sin(dLat / 2) * math.sin(dLat / 2) + \
            math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * \
            math.sin(dLon / 2) * math.sin(dLon / 2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c
        return distance

    def get_simulated_environmental_conditions(self, site_id, severity='normal'):
        """Simulates environmental conditions for a given geological site."""
        base_conditions = {
            'GA_RKI': 'Clear skies, -5°C, low avalanche risk, stable rockfall.',
            'GS_GTL': 'Partly cloudy, 18°C, moderate rain chance, stable ground.',
            'GD_SLF': 'Sunny, 40°C, high sandstorm risk, very dry.',
            'GR_RFN': 'Overcast, 25°C, heavy rainfall expected, high humidity.',
            'GM_OCN': 'Underwater currents normal, seismic activity low, water temp 4°C.',
            'AR_ICE': 'Blizzard conditions, -30°C, high whiteout risk, unstable ice.',
            'VL_JPN': 'Light ashfall, 15°C, moderate seismic tremor, active fumaroles.',
            'RS_SCL': 'Humid, 28°C, occasional sinkhole activity, dense vegetation.',
            'MZ_MID': 'Cloudy, 10°C, light snow, frozen ground.',
            'DS_SWM': 'Clear skies, 30°C, flash flood warning, rocky terrain.',
            'CR_GRN': 'Cold, -20°C, potential crevasse danger, ice flow normal.'
        }
        conditions = base_conditions.get(site_id.upper(), 'Unknown environmental conditions.')
        if severity == 'bad':
            conditions += ' ***SEVERE HAZARD ALERT! Operations suspended.***'
        elif severity == 'moderate':
            conditions += ' Moderate challenges reported. Proceed with caution.'
        return conditions

# --- Agent Service Class ---
class AgentService:
    def __init__(self, client, model_name, agent_roles):
        self.client = client
        self.model_name = model_name
        self.agent_roles = agent_roles

    def get_response(self, agent_name, user_prompt, conversation_history=None):
        """Gets an LLM response, acting as a specific agent, using the Mistral API."""
        system_message = self.agent_roles.get(agent_name, "You are a helpful AI assistant.")
        messages = [{"role": "system", "content": system_message}]
        if conversation_history:
            messages.extend(conversation_history)
        messages.append({"role": "user", "content": user_prompt})
        print(f"\n [Calling LLM as {agent_name} with prompt]: {user_prompt[:100]}...")
        try:
            chat_response = self.client.chat.complete(
                model=self.model_name,
                messages=messages,
            )
            return chat_response.choices[0].message.content
        except Exception as e:
            print(f"Error calling Mistral LLM for {agent_name}: {e}")
            return f"Error: {agent_name} could not provide a response due to an API error."

# --- Survey Class (representing a geological survey's state) ---
class GeologicalSurvey:
    def __init__(self, survey_id, origin_site_id, target_site_id, equipment_type):
        self.survey_id = survey_id
        self.origin_site_id = origin_site_id
        self.target_site_id = target_site_id
        self.equipment_type = equipment_type
        self.status = "Planned" # e.g., Planned, Active, Completed, Suspended, Cancelled
        self.current_plan = {} # Stores the detailed plan
        self.plan_history = [] # To keep track of re-planning events
        self.last_update_time = datetime.now()
        # Add attributes for new agents
        self.stakeholder_info = "Not yet gathered"
        self.personnel_info = "Not yet assigned"
        self.equipment_readiness = "Not yet checked"

    def update_plan(self, new_plan_details, reason="Initial Planning"):
        """Updates the current survey plan and logs it."""
        self.current_plan = new_plan_details
        self.plan_history.append({"timestamp": datetime.now(), "reason": reason, "plan": new_plan_details})
        self.last_update_time = datetime.now()
        print(f"Survey {self.survey_id}: Plan updated ({reason})")

    def get_status_report(self):
        """Generates a summary of the survey's current status and plan."""
        report = f"--- Survey {self.survey_id} Status Report ---\n"
        report += f"Route: {self.origin_site_id} to {self.target_site_id} with {self.equipment_type}\n"
        report += f"Status: {self.status}\n"
        if self.current_plan:
            report += f"Estimated Operational Duration: {self.current_plan.get('operational_duration_hours', 'N/A')} hours\n"
            report += f"Estimated Resources Required: {self.current_plan.get('resources_required', 'N/A')} units\n"
            report += f"Origin Environmental Conditions: {self.current_plan.get('origin_env_conditions', 'N/A')}\n"
            report += f"Target Environmental Conditions: {self.current_plan.get('target_env_conditions', 'N/A')}\n"
            report += f"Estimated Deployment Arrival: {self.current_plan.get('estimated_arrival_time', 'N/A')}\n"
            report += f"Stakeholder Info: {self.stakeholder_info}\n"
            report += f"Personnel: {self.personnel_info}\n"
            report += f"Equipment Readiness: {self.equipment_readiness}\n"
        return report

# --- ESOCC Controller Class ---
class ESOCC_Controller:
    def __init__(self, mistral_client, site_data, equipment_data, agent_roles, model_name):
        self.tool_service = ToolService()
        self.agent_service = AgentService(mistral_client, model_name, agent_roles)
        self.site_data = site_data
        self.equipment_data = equipment_data
        self.surveys = {} # Dictionary to store GeologicalSurvey objects, keyed by survey_id
        self.survey_id_counter = 0

    def create_survey(self, origin_site_id, target_site_id, equipment_type):
        """Creates a new survey entry and initiates planning."""
        self.survey_id_counter += 1
        survey_id = f"GS{self.survey_id_counter:04d}-{equipment_type.replace(' ','').replace('(','').replace(')','')}"
        new_survey = GeologicalSurvey(survey_id, origin_site_id, target_site_id, equipment_type)
        self.surveys[survey_id] = new_survey
        print(f"\n [ESOCC] Created new survey: {survey_id}")
        return survey_id

    def plan_survey_initial(self, survey_id, env_severity='normal'):
        """
        Conducts initial comprehensive geological survey planning for a given survey ID.
        Updates the state of a GeologicalSurvey object.
        """
        survey = self.surveys.get(survey_id)
        if not survey:
            print(f"Error: Survey {survey_id} not found.")
            return False

        orig_site_id = survey.origin_site_id
        target_site_id = survey.target_site_id
        equip_type = survey.equipment_type

        print(f"\n--- Initiating Initial Planning for Survey {survey_id} ({orig_site_id} to {target_site_id} with {equip_type}) ---")

        # Initial Data Gathering & Validation
        orig_site_data = self.site_data.get(orig_site_id.upper())
        target_site_data = self.site_data.get(target_site_id.upper())
        equipment_data_obj = self.equipment_data.get(equip_type.upper())

        if not orig_site_data or not target_site_data or not equipment_data_obj:
            print(f"Error: Invalid site IDs or equipment type for survey {survey_id}.")
            survey.status = "Planning Failed"
            return False

        print("\n[ESOCC Step 1: Initial Data Gathering & Validation]")
        print(f" Origin Site: {orig_site_data['name']} ({orig_site_id})")
        print(f" Target Site: {target_site_data['name']} ({target_site_id})")
        print(f" Equipment: {equip_type} (Deployment Speed: {equipment_data_obj['speed_deployment_kmh']} km/h, Resource Rate: {equipment_data_obj['resource_consumption_rate_units_hr']} units/hr, Range: {equipment_data_obj['operational_range_km']} km)")

        survey_plan_details = {} # Temporary dict to collect agent responses

        # Step-by-step orchestration using conceptual agents
        # Agent: Survey Request Processor
        print("\n [ESOCC Step 2: Survey Request Processor Processing Request]")
        user_request_prompt = (
            f"The research team needs to plan a geological survey from {orig_site_id} to {target_site_id} using a {equip_type}. "
            "A comprehensive operational plan considering all logistical and scientific factors is required. "
            f"The initial assumption for target site environmental conditions is '{env_severity}'. "
            "Summarize the core request and identify initial key operational data points needed for the plan."
        )
        response_content = self.agent_service.get_response('survey_request_processor', user_request_prompt)
        survey_plan_details['initial_request_processing'] = response_content

        # Agent: Equipment Resources Agent
        print("\n[ESOCC Step 3: Equipment Resources Agent Retrieving Data]")
        equipment_prompt = f"Provide detailed performance and availability data for a {equip_type} from the research fleet, including its typical deployment speed, resource consumption rate, and maximum operational range."
        response_content = self.agent_service.get_response('equipment_resources_agent', equipment_prompt)
        survey_plan_details['equipment_performance'] = response_content

        # Agent: Site Operations Agent
        print("\n[ESOCC Step 4: Site Operations Agent Retrieving Data]")
        site_ops_prompt = f"Provide comprehensive operational details for {orig_site_id} and {target_site_id} geological sites, including their full names, coordinates, and relevant terrain/access information for {equip_type} operations."
        response_content = self.agent_service.get_response('site_ops_agent', site_ops_prompt)
        survey_plan_details['site_info'] = response_content

        # Agent: Survey Planner Agent
        print("\n[ESOCC Step 5: Survey Planner Agent Calculating Route]")
        distance = self.tool_service.calculate_distance(orig_site_data['lat'], orig_site_data['lon'],
                                                       target_site_data['lat'], target_site_data['lon'])
        operational_duration_hours = distance / equipment_data_obj['speed_deployment_kmh']
        route_prompt = (
            f"The direct deployment distance between {orig_site_data['name']} and {target_site_data['name']} is {distance:.2f} km. "
            f"For a {equip_type} with a deployment speed of {equipment_data_obj['speed_deployment_kmh']} km/h, the estimated operational duration is {operational_duration_hours:.2f} hours. "
            "Based on this, describe an optimal survey path, considering geological features, terrain, and any general logistical requirements."
        )
        response_content = self.agent_service.get_response('survey_planner_agent', route_prompt)
        survey_plan_details['route_calculation'] = response_content
        survey_plan_details['calculated_distance'] = distance
        survey_plan_details['estimated_operational_duration_hours'] = operational_duration_hours

        # Agent: Geohazard Conditions Desk
        print("\n [ESOCC Step 6: Geohazard Conditions Desk Getting Conditions]")
        origin_env_data = self.tool_service.get_simulated_environmental_conditions(orig_site_id, 'normal')
        origin_env_prompt = f"Analyze the current and forecasted environmental and geohazard conditions at {orig_site_id}: '{origin_env_data}'. Provide operational implications for initial mobilization and site setup."
        response_content = self.agent_service.get_response('geohazard_conditions_desk', origin_env_prompt)
        survey_plan_details['origin_env_conditions'] = response_content

        # Agent: Target Environmental Conditions Desk
        print("\n[ESOCC Step 7: Target Environmental Conditions Desk Getting Conditions]")
        target_env_data = self.tool_service.get_simulated_environmental_conditions(target_site_id, env_severity)
        target_env_prompt = f"Analyze the current and forecasted environmental conditions at {target_site_id}: '{target_env_data}'. Discuss operational implications for survey execution, data collection, and potential hazards."
        response_content = self.agent_service.get_response('target_env_conditions_desk', target_env_prompt)
        survey_plan_details['target_env_conditions'] = response_content

        # Agent: En-Route Logistics & Environmental Desk
        print("\n[ESOCC Step 8: En-Route Logistics & Environmental Desk Analyzing Transit Conditions]")
        enroute_proxy_site = random.choice(list(self.site_data.keys()))
        enroute_env_data = self.tool_service.get_simulated_environmental_conditions(enroute_proxy_site, random.choice(['normal', 'moderate']))
        enroute_env_prompt = f"Analyze potential en-route logistical and environmental conditions for transit from {orig_site_id} to {target_site_id}, considering simulated conditions near {enroute_proxy_site}: '{enroute_env_data}'. Identify any significant challenges (e.g., road closures, extreme weather, permit delays)."
        response_content = self.agent_service.get_response('enroute_logistics_env_desk', enroute_env_prompt)
        survey_plan_details['enroute_env_logistics'] = response_content

        # Agent: Safety & Environmental Compliance Officer
        print("\n[ESOCC Step 9: Safety & Environmental Compliance Officer Checking Regulations]")
        compliance_prompt = f"For a survey from {orig_site_id} to {target_site_id} using a {equip_type}, what key general safety, environmental protection, and land access regulatory considerations should be checked and complied with?"
        response_content = self.agent_service.get_response('safety_env_compliance_officer', compliance_prompt)
        survey_plan_details['safety_env_compliance'] = response_content

        # Agent: Resource & Logistics Optimiser
        print("\n[ESOCC Step 10: Resource & Logistics Optimiser Calculating Requirements]")
        resources_required_calculated = operational_duration_hours * equipment_data_obj['resource_consumption_rate_units_hr']
        resource_load_prompt = (
            f"Calculate the estimated resources (e.g., fuel, water, specialized materials) required for a {equip_type} operating for duration: {operational_duration_hours:.2f} hours, "
            f"with a consumption rate of {equipment_data_obj['resource_consumption_rate_units_hr']} units/hour. "
            f"The calculated base resource need is {resources_required_calculated:.2f} units. "
            "Provide critical guidance on equipment load balancing, supply chain management, and contingency resource allocation."
        )
        response_content = self.agent_service.get_response('resource_logistics_optimiser', resource_load_prompt)
        survey_plan_details['resource_logistics'] = response_content
        survey_plan_details['estimated_resources_required'] = resources_required_calculated

        # Agent: Contingency Response Specialist
        print("\n[ESOCC Step 11: Contingency Response Specialist Developing Alternates]")
        alternate_site_id = 'RS_SCL' # Example: In a real system, this would be dynamically chosen based on survey parameters
        alternate_env_data = self.tool_service.get_simulated_environmental_conditions(alternate_site_id, 'normal')
        contingency_prompt = (
            f"Given the target site conditions at {target_site_id} are '{env_severity}' ({survey_plan_details.get('target_env_conditions', 'N/A')}), "
            f"and considering a {equip_type} (operational range: {equipment_data_obj['operational_range_km']} km), "
            f"and a potential alternate research site {alternate_site_id} with conditions '{alternate_env_data}', "
            "suggest suitable alternate strategies or fallback locations and provide actionable advice on general contingency planning (e.g., emergency protocols, communication, resource reallocation)."
        )
        response_content = self.agent_service.get_response('contingency_response_specialist', contingency_prompt)
        survey_plan_details['contingency_response'] = response_content

        # Agent: Stakeholder Liaison
        print("\n[ESOCC Step 12: Stakeholder Liaison Assessing Engagement]")
        stakeholder_prompt = f"For survey {survey_id} ({orig_site_id} to {target_site_id} with {equip_type}), provide insights into managing relationships with local communities, indigenous groups, and other stakeholders. Include communication strategies and potential impacts."
        response_content = self.agent_service.get_response('stakeholder_liaison', stakeholder_prompt)
        survey_plan_details['stakeholder_info'] = response_content
        survey.stakeholder_info = response_content # Update survey object

        # Agent: Personnel Scheduler
        print("\n[ESOCC Step 13: Personnel Scheduler Optimizing Team Assignment]")
        personnel_prompt = f"For survey {survey_id} ({orig_site_id} to {target_site_id} with {equip_type}, estimated operational duration {operational_duration_hours:.2f} hours), provide team availability, duty time limitations, rest requirements, and optimal team pairings. Ensure compliance with labor laws and field safety protocols."
        response_content = self.agent_service.get_response('personnel_scheduler', personnel_prompt)
        survey_plan_details['personnel_info'] = response_content
        survey.personnel_info = response_content # Update survey object

        # Agent: Equipment Readiness Coordinator
        print("\n [ESOCC Step 14: Equipment Readiness Coordinator Confirming Readiness]")
        equipment_readiness_prompt = f"For {equip_type} assigned to survey {survey_id}, report on its current maintenance status, recent servicing, potential mechanical issues, and required calibrations/inspections to confirm its readiness for deployment."
        response_content = self.agent_service.get_response('equipment_readiness_coordinator', equipment_readiness_prompt)
        survey_plan_details['equipment_readiness'] = response_content
        survey.equipment_readiness = response_content # Update survey object

        # Final Synthesis by the Geo-Operational Director
        print("\n[ESOCC Step 15: Geo-Operational Director Final Synthesis of Operational Plan]")
        final_synthesis_prompt = (
            "As the ESOCC Geo-Operational Director, synthesize the following information provided by various specialized agents into a cohesive, comprehensive, and actionable geological survey plan. "
            f"Ensure it's well-structured with clear headings and covers all critical aspects for a survey from {orig_site_id} to {target_site_id} with a {equip_type}. "
            "Include estimated operational duration and resources, and a concise summary of all key ESOCC findings and recommendations.\n\n"
            f"**Survey Request Summary:** {survey_plan_details.get('initial_request_processing', 'N/A')}\n\n"
            f"**Equipment Resources Assessment:** {survey_plan_details.get('equipment_performance', 'N/A')}\n\n"
            f"**Site Operational Data:** {survey_plan_details.get('site_info', 'N/A')}\n\n"
            f"**Survey Path & Logistics:** {survey_plan_details.get('route_calculation', 'N/A')}\n\n"
            f"**Origin Environmental Brief:** {survey_plan_details.get('origin_env_conditions', 'N/A')}\n\n"
            f"**Target Environmental Brief:** {survey_plan_details.get('target_env_conditions', 'N/A')}\n\n"
            f"**En-Route Environmental & Logistics Outlook:** {survey_plan_details.get('enroute_env_logistics', 'N/A')}\n\n"
            f"**Safety & Environmental Compliance Check:** {survey_plan_details.get('safety_env_compliance', 'N/A')}\n\n"
            f"**Resource & Logistics Optimisation:** {survey_plan_details.get('resource_logistics', 'N/A')}\n\n"
            f"**Contingency & Response Plan:** {survey_plan_details.get('contingency_response', 'N/A')}\n\n"
            f"**Stakeholder Engagement Brief:** {survey_plan_details.get('stakeholder_info', 'N/A')}\n\n"
            f"**Personnel Assignment & Compliance:** {survey_plan_details.get('personnel_info', 'N/A')}\n\n"
            f"**Equipment Readiness Status:** {survey_plan_details.get('equipment_readiness', 'N/A')}\n\n"
            f"**Calculated Direct Deployment Distance:** {distance:.2f} km\n"
            f"**Estimated Operational Duration:** {round(operational_duration_hours * 60)} minutes\n"
            f"**Estimated Resources Required:** {resources_required_calculated:.2f} units\n"
            f"**Estimated Deployment Arrival Time:** {(datetime.now() + timedelta(minutes=round(operational_duration_hours * 60))).strftime('%Y-%m-%d %H:%M')} (local time)\n"
        )
        final_llm_plan_details = self.agent_service.get_response('geo_operational_director', final_synthesis_prompt)

        final_plan_summary = {
            'origin': orig_site_data['name'],
            'target': target_site_data['name'],
            'equipment': equip_type,
            'distance_km': f"{distance:.2f}",
            'operational_duration_minutes': round(operational_duration_hours * 60),
            'resources_required': f"{resources_required_calculated:.2f}",
            'origin_env_conditions': survey_plan_details.get('origin_env_conditions', 'N/A'),
            'target_env_conditions': survey_plan_details.get('target_env_conditions', 'N/A'),
            'llm_plan_details': final_llm_plan_details,
            'estimated_arrival_time': (datetime.now() + timedelta(minutes=round(operational_duration_hours * 60))).strftime('%Y-%m-%d %H:%M')
        }
        survey.update_plan(final_plan_summary, reason="Initial Planning Completed")
        survey.status = "Planned"
        return True

    def monitor_surveys(self):
        """Simulates ongoing monitoring of active geological surveys by the ESOCC."""
        print("\n[ESOCC] Monitoring active surveys...")
        for survey_id, survey in self.surveys.items():
            if survey.status == "Planned" or survey.status == "Active":
                print(f" Monitoring {survey.survey_id} Status: {survey.status}")
                # Example: Check for stale plan (e.g., plan older than 24 hours)
                if (datetime.now() - survey.last_update_time).total_seconds() > 3600 * 24:
                    print(f" Survey {survey_id}: Plan is stale, considering re-planning.")
                    # In a real ESOCC, this might trigger an alert to an operator
                    # or automatically initiate a minor replan.
                    # self.replan_survey(survey_id, reason="Stale plan due to time lapse")

    def replan_survey(self, survey_id, new_env_severity=None, reason="Manual Re-planning Triggered"):
        """
        Triggers a re-planning cycle for a specific survey,
        e.g., due to updated environmental conditions, equipment issues, or site access changes.
        """
        survey = self.surveys.get(survey_id)
        if not survey:
            print(f"Error: Survey {survey_id} not found for re-planning.")
            return False

        print(f"\n--- Initiating Re-planning for Survey {survey_id} ({reason}) ---")
        current_env_severity = new_env_severity if new_env_severity else 'normal'
        success = self.plan_survey_initial(survey_id, env_severity=current_env_severity)
        if success:
            survey.status = "Planned (Re-planned)"
            print(f"Re-planning for Survey {survey_id} completed successfully.")
        else:
            survey.status = "Re-planning Failed"
            print(f"Re-planning for Survey {survey_id} failed.")
        return success

    def get_survey_report(self, survey_id):
        """Retrieves a comprehensive report for a specific geological survey."""
        survey = self.surveys.get(survey_id)
        if survey:
            print(survey.get_status_report())
            if survey.current_plan and survey.current_plan.get('llm_plan_details'):
                print("\n-- Full LLM Synthesized Geo-Operational Plan Details --")
                print(survey.current_plan['llm_plan_details'])
        else:
            print(f"Survey {survey_id} not found.")


# --- Main Execution Block (simulating ESOCC operations) ---
if __name__ == "__main__":
    # Ensure Mistral API client is initialized
    try:
        # import colab_env # This line might cause issues if not in Colab
        api_key = os.environ.get("MISTRAL_API_KEY")
        if not api_key:
            raise ValueError("MISTRAL_API_KEY environment variable not set. Please set your Mistral API key before running this script.")
        client = Mistral(api_key=api_key)
    except (ImportError, ValueError) as e:
        print(f"Error initializing Mistral client: {e}. Please ensure colab-env is installed and MISTRAL_API_KEY is set.")
        exit()

    MISTRAL_MODEL = "open-mixtral-8x22b" # Using the same model as the original

    # Initialize the ESOCC Controller with the new agent roles
    esocc_center = ESOCC_Controller(client, GEOLOGICAL_SITE_DATA, EQUIPMENT_DATA, ESOCC_AGENT_ROLES, MISTRAL_MODEL)

    # Scenario 1: Initial Survey Planning for a new geological expedition
    print("\n-- ESOCC Scenario 1: Processing New Survey Request for Geo-Operational Planning ---")
    survey1_id = esocc_center.create_survey("GS_GTL", "VL_JPN", "SEISMIC VESSEL")
    esocc_center.plan_survey_initial(survey1_id)
    esocc_center.get_survey_report(survey1_id)

    # Scenario 2: Simulate an Environmental Change-driven Re-planning (Operational Adjustment)
    print("\n\n--- ESOCC Scenario 2: Operational Adjustment due to Environmental Change ---")
    new_target_env_severity = random.choice(['moderate', 'bad'])
    print(f" [ESOCC] Detecting a change to '{new_target_env_severity}' conditions at target site for Survey {survey1_id}. Initiating operational re-planning.")
    esocc_center.replan_survey(survey1_id, new_env_severity=new_target_env_severity, reason=f"Dynamic environmental update to '{new_target_env_severity}' at VL_JPN")
    esocc_center.get_survey_report(survey1_id)

    # Scenario 3: Add another survey to the operational schedule
    print("\n\n-- ESOCC Scenario 3: Adding another Survey to Operational Schedule")
    survey2_id = esocc_center.create_survey("GA_RKI", "AR_ICE", "DRILLING RIG (LARGE)")
    esocc_center.plan_survey_initial(survey2_id)
    esocc_center.get_survey_report(survey2_id)

    # Scenario 4: ESOCC Global Monitoring Check
    print("\n\n--- ESOCC Scenario 4: Global Survey Monitoring Check ---")
    esocc_center.monitor_surveys()


-- ESOCC Scenario 1: Processing New Survey Request for Geo-Operational Planning ---

 [ESOCC] Created new survey: GS0001-SEISMICVESSEL

--- Initiating Initial Planning for Survey GS0001-SEISMICVESSEL (GS_GTL to VL_JPN with SEISMIC VESSEL) ---

[ESOCC Step 1: Initial Data Gathering & Validation]
 Origin Site: Great Lakes Basin Survey Zone (GS_GTL)
 Target Site: Japanese Volcanic Arc Study Area (VL_JPN)
 Equipment: SEISMIC VESSEL (Deployment Speed: 20 km/h, Resource Rate: 1000 units/hr, Range: 10000 km)

 [ESOCC Step 2: Survey Request Processor Processing Request]

 [Calling LLM as survey_request_processor with prompt]: The research team needs to plan a geological survey from GS_GTL to VL_JPN using a SEISMIC VESSEL. A ...

[ESOCC Step 3: Equipment Resources Agent Retrieving Data]

 [Calling LLM as equipment_resources_agent with prompt]: Provide detailed performance and availability data for a SEISMIC VESSEL from the research fleet, inc...

[ESOCC Step 4: Site Operations Agent Retrieving

## MISTRAL API - AGENTIC